In [ ]:
import langchain
import chromadb

In [ ]:
from langchain.llms import OpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from pathlib import Path
from langchain.chat_models import ChatOpenAI
from langchain.chains import QAGenerationChain
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.chains import ConversationalRetrievalChain
from langchain.chains import ChatVectorDBChain
from langchain.evaluation.qa import QAEvalChain
from langchain.evaluation.qa import ContextQAEvalChain
from langchain.llms import OpenAIChat


In [ ]:
# pip install tiktoken
# pip install openai

In [ ]:
import os

[x for x in os.walk("chroma_db")]

In [ ]:
from glob import glob
glob("chroma_db/*/", recursive = True)

In [ ]:
[f.path for f in os.scandir("chroma_db") if f.is_dir()]

In [ ]:
import os

persist_directory = "chroma_db"
collection_name = "test_PDF_2"

os.path.exists(f"{persist_directory}/{collection_name}")

In [ ]:
session_state = []

existing_collection_names = [db["collection_name"] for db in session_state]

In [ ]:
existing_collection_names

In [ ]:
# pip install gpt4all

from langchain.llms import GPT4All

local_path = ("llm/gpt4all-converted.bin")

llm = GPT4All(model = local_path, n_ctx=2000)

In [ ]:
import langchain

langchain.__version__

In [ ]:
question = "What NFL team won the Super Bowl in the year Justin Bieber was born?"

llm.generate([question])

In [ ]:
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import RetrievalQA
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

collection_name = "test_PDF_2"
persist_directory = "chroma_db"

embedding_function = HuggingFaceEmbeddings()

database_search = Chroma(collection_name = collection_name,
                         persist_directory=f"{persist_directory}/{collection_name}", 
                         embedding_function=embedding_function)

chain = RetrievalQA.from_chain_type(llm=llm,
                                    chain_type="stuff",
                                    retriever = database_search.as_retriever(search_type="similarity", search_kwargs={"k":1}),
                                    return_source_documents = True)

In [ ]:
query = "What is the importance of lookahead in model validation?"


In [ ]:
result

In [ ]:
# pip install pygpt4all==1.0.1
# pip install pygptj==1.0.10
# pip install pyllamacpp==1.0.6
# pip install langchain==0.0.142

In [ ]:
# from langchain.llms import BaseLLM

# llm = OpenAI(max_tokens=)

In [ ]:
from langchain.agents import initialize_agent, AgentExecutor

In [ ]:
from langchain.llms import GPT4All, OpenAI


In [ ]:
from langchain.agents import Tool
from langchain.agents import AgentType
from langchain.memory import ConversationBufferMemory
from langchain import OpenAI
from langchain.utilities import SerpAPIWrapper
from langchain.agents import initialize_agent
from langchain.llms import OpenAIChat

In [ ]:
import openai

In [ ]:
def list_all_models():
    model_list = openai.Model.list()['data']
    model_ids = [x['id'] for x in model_list]
    model_ids.sort()
    print(model_ids)

list_all_models()

In [ ]:
from langchain import OpenAI

In [ ]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

embedding_function = HuggingFaceEmbeddings()

In [ ]:
embedding_function.model_name

In [1]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.agents import initialize_agent, Tool, AgentExecutor, AgentType, load_tools
from langchain.memory import ConversationBufferMemory


In [2]:

chunk_size = 200
chunk_overlap = 10

loader = PyPDFLoader("data/messages.pdf")
documents = loader.load()

text_splitter = CharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap,
                                      separator = " ") 

documents = text_splitter.split_documents(documents)

embedding_function = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-mpnet-base-v2")

database = Chroma.from_documents(documents,
                                 embedding = embedding_function)

retriever = database.as_retriever(search_type="similarity",search_kwargs={"k":3})

llm = ChatOpenAI(model = 'gpt-3.5-turbo', temperature = 0.7, openai_api_key = "")

chain = RetrievalQA.from_chain_type(llm = llm,
                                    chain_type = "stuff", # chain_type: specifying how the RetrievalQA should pass the chunks into LLM
                                    retriever = retriever)




c:\Users\angus\Anaconda3\envs\langchain-test\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
tools = [Tool(name="messages",
                   func=chain.run,
                   description="A chat history for MyDrumSchool news. Useful for finding out about the latest happenings in MyDrumSchool.")]

In [4]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

agent = initialize_agent(tools, llm, 
                        agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION, 
                        verbose=True, memory=memory, handle_parsing_errors=True)

In [ ]:
import wandb

In [ ]:
wandb.login(key="")

In [5]:
PROJECT = "langchain_qa_app_test"
wandb.init(project=PROJECT, job_type="generation")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: angussaw39. Use `wandb login --relogin` to force relogin


In [6]:
import os
os.environ["LANGCHAIN_WANDB_TRACING"] = "true"

In [7]:
#agent({"input":"Hello!"})["output"]
agent.run("Hello!")



> Entering new AgentExecutor chain...
{
    "action": "Final Answer",
    "action_input": "Hello! How can I assist you today?"
}

> Finished chain.


'Hello! How can I assist you today?'

In [10]:
output = agent.run("Hello!")

wandb: Streaming LangChain activity to W&B at https://wandb.ai/angussaw39/langchain_QA_app/runs/epz5hgc4
wandb: `WandbTracer` is currently in beta.
wandb: Please report any issues to https://github.com/wandb/wandb/issues with the tag `langchain`.




> Entering new AgentExecutor chain...
{
    "action": "Final Answer",
    "action_input": "Hello! How can I assist you today?"
}

> Finished chain.


'Hello! How can I assist you today?'

In [11]:
output

'Hello! How can I assist you today?'

In [8]:
#agent({"input":"As a student of MyDrumSchool, how can I sign up for Rockschool exams?"})["output"]
agent.run("As a student of MyDrumSchool, how can I sign up for Rockschool exams?")




> Entering new AgentExecutor chain...
Could not parse LLM output: To sign up for Rockschool exams as a student of MyDrumSchool, you can follow these steps:

1. Visit the MyDrumSchool website and navigate to the Rockschool exams section.
2. Look for information on exam dates, fees, and registration deadlines.
3. Fill out the registration form with your personal details and select the Rockschool exam level you wish to take.
4. Pay the required exam fees through the designated payment method.
5. Once your registration is processed, you will receive confirmation of your exam booking along with further instructions.

If you need more specific information or assistance with the sign-up process, I recommend reaching out to the MyDrumSchool team directly for guidance. They will be able to provide you with the most up-to-date and accurate information regarding Rockschool exams and the sign-up procedure.
Observation: Invalid or incomplete response
Thought:{
    "action": "Final Answer",
    "a

'To sign up for Rockschool exams as a student of MyDrumSchool, you can follow these steps:\n\n1. Visit the MyDrumSchool website and navigate to the Rockschool exams section.\n2. Look for information on exam dates, fees, and registration deadlines.\n3. Fill out the registration form with your personal details and select the Rockschool exam level you wish to take.\n4. Pay the required exam fees through the designated payment method.\n5. Once your registration is processed, you will receive confirmation of your exam booking along with further instructions.\n\nIf you need more specific information or assistance with the sign-up process, I recommend reaching out to the MyDrumSchool team directly for guidance. They will be able to provide you with the most up-to-date and accurate information regarding Rockschool exams and the sign-up procedure.'

In [12]:
wandb.finish()

In [22]:
agent({"input":"Based on the technical skills and years of experience, find two suitable data scientist roles in Singapore for Angus"})["output"]



> Entering new AgentExecutor chain...
{
    "action": "Search",
    "action_input": "data scientist roles in Singapore"
}
Observation: https://sg.linkedin.com/jobs/data-scientist-jobs
Thought:{
    "action": "Final Answer",
    "action_input": "Based on the technical skills and years of experience, two suitable data scientist roles in Singapore for Angus could be 'Data Scientist' and 'Senior Data Scientist'."
}

> Finished chain.


"Based on the technical skills and years of experience, two suitable data scientist roles in Singapore for Angus could be 'Data Scientist' and 'Senior Data Scientist'."

In [37]:
query = "As a student of MyDrumSchool, how can I sign up for Rockschool exams?"

similar_sentences = database.similarity_search_with_score(query = query,
                                                          k = 4)

similar_sentences

[(Document(page_content='A short workshop and updates on the Rockschool examinations.\nThere will be live performances by our Educators.\nMore details will be released in your MDS Room App ������\n@mydrumschool\n16 June 2020\nMy', metadata={'source': 'data/messages.pdf', 'page': 31}),
  0.6857510209083557),
 (Document(page_content='18.8 KB�Rockschool Exams Start tomorrow!\nAll the best to the candidates for Rockschool Drumkit Exams Tour A\n2022!\nPlease ensure to arrive 15 minutes before your stipulated exam time.\nExam Venue:\nRS', metadata={'source': 'data/messages.pdf', 'page': 69}),
  0.7530626654624939),
 (Document(page_content='Rockschool Drumkit Exams Tour B\n2022!\nPlease ensure to arrive 15 minutes before your stipulated exam time.\nExam Venue:\nRS Event Center\nBlk 51 Old Airport Road\n#02-56\nSingapore 390051\nCompulsory items', metadata={'source': 'data/messages.pdf', 'page': 74}),
  0.7836322784423828),
 (Document(page_content='Exams next week.\n@mydrumschool\nhttps://yout

In [38]:
chain.run({"query":query})

"To sign up for Rockschool exams as a student of MyDrumSchool, you can follow these steps:\n\n1. Check for any announcements or updates on Rockschool examinations in your MDS Room App or any other communication channels from MyDrumSchool.\n\n2. Look for information regarding the upcoming Rockschool exams, including the dates, venues, and any specific requirements or instructions.\n\n3. If you are interested in taking the exams, ensure that you meet the eligibility criteria and have prepared the necessary repertoire and skills.\n\n4. Contact MyDrumSchool or the designated exam coordinator to express your interest in signing up for the Rockschool exams. They will provide you with the necessary registration forms and guide you through the registration process.\n\n5. Fill out the registration forms accurately and submit them along with any required documents or fees within the specified deadline.\n\n6. Once your registration is confirmed, you will receive further instructions regarding the

In [35]:
query = "How can I rent a studio for practice?"

similar_sentences = database.similarity_search_with_score(query = query,
                                                          k = 3)

similar_sentences

[(Document(page_content='now use your Lesson Credits to convert them to Studio Rental\nCredits (SRC). Use your SRC to make a rental booking with us. Available\nstudios of up to the next 7 days will be displayed.\nHere is a video', metadata={'source': 'data/messages.pdf', 'page': 92}),
  0.5762842893600464),
 (Document(page_content='We will be opening a total of 4 fully-fitted drum studios with an\nadditional 200 student intake. We have also hired 2 more full-time staff\nto cater to the overflow of demand.\nStudios will feature', metadata={'source': 'data/messages.pdf', 'page': 3}),
  1.0864653587341309),
 (Document(page_content='for the inconvenience caused.\n12 March 2023\nMy Drum School News\n��� ���� Introducing Studio Rental ��� ����\nWe will be piloting Studio Rental Booking through the MDS Room App.My Drum School', metadata={'source': 'data/messages.pdf', 'page': 91}),
  1.1508418321609497)]

In [36]:
chain.run({"query":query})

'To rent a studio for practice, you can follow these steps:\n\n1. Make sure you have enough Lesson Credits in your account.\n2. Convert your Lesson Credits to Studio Rental Credits (SRC) through the MDS Room App.\n3. Use your SRC to make a rental booking for a studio.\n4. The available studios for the next 7 days will be displayed on the app.\n5. Select a studio and book it for your desired date and time.\n6. Confirm your booking and make the necessary payment, if required.\n7. Arrive at the studio on the scheduled date and time for your practice session.\n\nPlease note that the availability of studios may vary and it is advisable to book in advance to secure your preferred time slot.'

In [ ]:
import pandas as pd

texts = []
scores = []


for document in similar_sentences:
    text = document[0].page_content
    texts.append(text)
    score = document[1]
    scores.append(score)

similar_sentences_df = pd.DataFrame()
similar_sentences_df["text"] = texts
similar_sentences_df["score"] = scores



In [ ]:
pd.set_option('display.max_colwidth', None)
similar_sentences_df

In [ ]:
chain.combine_documents_chain.llm_chain.prompt.messages[0].prompt.template

In [ ]:
chain.combine_documents_chain.llm_chain.prompt.messages[1].prompt.template

In [ ]:
len(documents)

In [ ]:
documents